## Registering to Model Manager
This notebook exemplifies how to register a model developed in Workbench to Model Manager.

In [ ]:
import json
import requests 
import os
import sasviya 
from sasctl import Session, pzmm
from sasctl.services import model_repository as mr
import pickle 
import pandas as pd

In [ ]:
url = "https://innovationlab.demo.sas.com/"

def _connect_to_instance(refresh_token, verification):
	auth_url = f"{url}/SASLogon/oauth/token"
	# reading long-lived refresh token from txt file

	payload=f'grant_type=refresh_token&refresh_token={refresh_token}'
	headers = {
	'Accept': 'application/json',
	'Content-Type': 'application/x-www-form-urlencoded',
	'Authorization': 'Basic c2FzLmNsaTo=',
	}

	response = requests.request("POST", auth_url, headers=headers, data=payload, verify=verification)
	access_token = response.json()['access_token']
	return access_token

def _generate_access_token(auth_code, verification): 
	server = f"{url}/SASLogon/oauth/token"

	payload = f'grant_type=authorization_code&code={auth_code}'
	headers = {
		'Accept': 'application/json',
		'Content-Type': 'application/x-www-form-urlencoded',
		'Authorization': 'Basic c2FzLmNsaTo='
	}

	response = requests.request("POST", server, headers=headers, data=payload, verify=verification)

	# Parse the response text
	response_json = json.loads(response.text)

	# Get the refresh token
	refresh_token = response_json['refresh_token']
	return refresh_token

def get_connection(verify=False):
	if 'refresh_token.txt' not in [i.name for i in os.scandir()]:
		with open('refresh_token.txt', 'w') as file:
			file.write('test')
	if verify:
		os.environ['CAS_CLIENT_SSL_CA_LIST'] = 'innovation_lab.pem'
		verify = 'innovation_lab.pem'
	try:
		with open('refresh_token.txt', 'r') as token:
			refresh_token = token.read()
		access_token = _connect_to_instance(refresh_token, verify)

	except:
		print(f'{url}/SASLogon/oauth/authorize?client_id=sas.cli&response_type=code')
		auth_code = input(f'Please provide your access token by going to {url}/SASLogon/oauth/authorize?client_id=sas.cli&response_type=code:')
		refresh_token = _generate_access_token(auth_code, verify)
		with open('refresh_token.txt', 'w') as file:
			file.write(refresh_token)
		access_token = _connect_to_instance(refresh_token, verify)
		with open('access_token.txt', 'w') as file:
			file.write(access_token)
	print('Connected!')
	return access_token

In [ ]:
access_token = get_connection(verify=True)

In [ ]:
st = Session(url, token=access_token)
st

In [ ]:
project_name = "Live_SAS_WBWorkshop_Python"
repository_name = "DMRepository"

repository = mr.get_repository(repository_name)

project = mr.get_project(project_name)

if project == None:
    project = mr.create_project(project_name, repository)

### Register sasviya.ml Model

In [ ]:
model = sasviya.load_model('artifacts/gb_model.pkl')

model_params = {
    "name": "GB_Model_ViyaML",
    "projectId": project.id,
    "type": "ASTORE"
}

astore = mr.post(
    "/models",
    files={"files": ("model_export.astore", model.export())},
    data=model_params,
)

### Register Sk-Learn Model

Link to Tutorial: https://github.com/sassoftware/python-sasctl/blob/master/examples/pzmm_binary_classification_model_import.ipynb

In [ ]:
df = pd.read_csv('../../Data/cleaned_data/train_valid_test.csv')
train = df[df['_PartInd_']==1].reset_index(drop=True)
test = df[df['_PartInd_']==3].reset_index(drop=True)

numerical_features = ['CreditLineAge','DebtIncRatio','FICOScore','Inquiries6Mnths',
                      'LogAnnualInc','RevBalance','RevUtilization','Installment','InterestRate']
categorical_features = ['CreditPolicy','Delinquencies2Yrs','PublicRecord','Purpose']
features = numerical_features+categorical_features
target = 'Default'

with open('artifacts/data_prep.pkl', mode='rb') as f:
    data_prep = pickle.load(f)
x_train_prep = pd.DataFrame(data_prep.transform(train[features]), columns=features)
x_test_prep = pd.DataFrame(data_prep.transform(test[features]), columns=features)

with open('artifacts/sk_model.pkl', mode='rb') as f:
    sk_model = pickle.load(f)

In [ ]:
score_metrics = ["I_Default", "P_Default1", "P_Default0"]

def write_json_files(data, predict, target, path, prefix):    
    # Write input variable mapping to a json file
    pzmm.JSONFiles.write_var_json(input_data=data[predict], is_input=True, json_path=path)
    
    # Set output variables and assign an event threshold, then write output variable mapping
    output_var = pd.DataFrame(columns=score_metrics, data=[['1', 0.5, 0.5]]) # data argument includes example expected types for outputs
    pzmm.JSONFiles.write_var_json(output_var, is_input=False, json_path=path)
    
    # Write model properties to a json file
    pzmm.JSONFiles.write_model_properties_json(
        model_name=prefix, 
        target_variable=target, # Target variable to make predictions about (BAD in this case)
        target_values=["1", "0"], # Possible values for the target variable (1 or 0 for binary classification of BAD)
        json_path=path, 
        model_desc=f"Description for the {prefix} model.",
        model_algorithm="",
        modeler="faceru",
    )
    
    # Write model metadata to a json file so that SAS Model Manager can properly identify all model files
    pzmm.JSONFiles.write_file_metadata_json(model_prefix=prefix, json_path=path)

In [ ]:
mypath = 'artifacts/sk_registration'
prefix = 'SK_GB'
if not os.path.isdir(mypath):
   os.makedirs(mypath)

pzmm.PickleModel.pickle_trained_model(
        model_prefix=prefix,
        trained_model=sk_model,
        pickle_path=mypath
    )
write_json_files(x_train_prep, features, target, mypath, prefix)

In [ ]:
pzmm.ImportModel.import_model(
        model_files=mypath, # Where are the model files?
        model_prefix=prefix, # What is the model name?
        project=project_name, # What is the project name?
        input_data=x_train_prep, # What does example input data look like?
        predict_method=[sk_model.predict_proba, [int, int]], # What is the predict method and what does it return?
        score_metrics=score_metrics, # What are the output variables?
        overwrite_model=True, # Overwrite the model if it already exists?
        target_values=["0", "1"], # What are the expected values of the target variable?
        target_index=1, # What is the index of the target value in target_values?
        model_file_name=prefix + ".pickle", # How was the model file serialized?
        missing_values=False # Does the data include missing values?
    )